In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib
import mapclassify
import numpy as np
import shapely
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import hdbscan

In [ ]:
data_path = "C:/Users/jpg23/data/downtownrecovery/lehd_new_downtowns/"
df = gpd.read_file(data_path + "block_random_pts.geojson")

In [ ]:
def explore_pts(city_name):
    # filter out points with no lat/long, and filter to city
    df1 = df[(df['point_lon'].notna()) & (df['place']==city_name)].set_crs(4326)

    return(df1.explore(
        tiles="CartoDB positron",
        style_kwds=dict(opacity=.5, fillOpacity=.5)
    ))

In [ ]:
def create_clusters_tuned(cl_size, min_samp, ep, city_name):
    
    # filter out points with no lat/long, and filter to city
    df1 = df[(df['point_lon'].notna()) & (df['place']==city_name)]

    # standardize lat/long values
    X = StandardScaler().fit_transform(df1[['point_lon', 'point_lat']].copy())
    
    clusterer_tuned = hdbscan.HDBSCAN(min_cluster_size = cl_size,
                                      min_samples = min_samp,
                                      cluster_selection_epsilon = ep,
                                      core_dist_n_jobs=1)
    
    clusterer_tuned.fit(X)
    
    df1['cluster'] = clusterer_tuned.labels_
    df1.head()

    # Filter out non-clustered points, filter to city, and set CRS
    clusters_only_tuned = df1[df1["cluster"]!=-1].set_crs(4326)
    clusters_only_tuned['cluster'] = clusters_only_tuned['cluster'].astype(str)
    
    return(clusters_only_tuned)

In [ ]:
def create_map(cl_size, min_samp, ep, city_name):

    df = create_clusters_tuned(cl_size, min_samp, ep, city_name)
    
    return(df.explore(
        column="cluster",
        tiles="CartoDB positron",
        style_kwds=dict(opacity=.5, fillOpacity=.5),
        cmap='Spectral'
    ))

In [ ]:
def map_hull(cl_size, min_samp, ep, city_name, cluster_list, r):

    df = create_clusters_tuned(cl_size, min_samp, ep, city_name)

    one_cluster = df[df['cluster'].isin(cluster_list)][['geometry']]
    clust_buff = one_cluster.buffer(0.001)

    cluster_union = shapely.unary_union(clust_buff.geometry)

    hull = shapely.concave_hull(cluster_union, ratio = .8)
    hull_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[hull])

    m = one_cluster.explore(
        tiles="CartoDB positron",
        color = "red",
        style_kwds=dict(opacity=.5, fillOpacity=.5)
    )

    m = hull_gdf.explore(
            m = m,
            tiles="CartoDB positron",
            style_kwds=dict(opacity=.5, fillOpacity=.5)
        )

    folium.LayerControl().add_to(m)

    return(m)

# Albuquerque NM

In [ ]:
explore_pts('Albuquerque NM')

In [ ]:
create_map(50, 25, 0, 'Albuquerque NM')

In [ ]:
map_hull(50, 25, 0, 'Albuquerque NM', ['7'], 0.9)

# Atlanta GA

In [ ]:
explore_pts('Atlanta GA')

In [ ]:
create_map(100, 50, 0.04, 'Atlanta GA')

In [ ]:
map_hull(100, 50, 0.04, 'Atlanta GA', ['4', '6', '7'], .9)

# Austin TX

In [ ]:
explore_pts('Austin TX')

In [ ]:
create_map(40, 40, 0, 'Austin TX')

In [ ]:
map_hull(40, 40, 0, 'Austin TX', ['20'], 0.8)

# Bakersfield CA

In [ ]:
explore_pts('Bakersfield CA')

In [ ]:
create_map(25, 25, 0, 'Bakersfield CA')

In [ ]:
map_hull(25, 25, 0, 'Bakersfield CA', ['10'], 0.8)

# Baltimore MD

In [ ]:
explore_pts('Baltimore MD')

In [ ]:
create_map(50, 50, 0, 'Baltimore MD')

In [ ]:
map_hull(50, 50, 0, 'Baltimore MD', ['9'], 0.8)

# Boston MA

In [ ]:
explore_pts('Boston MA')

In [ ]:
create_map(30, 30, 0, 'Boston MA')

In [ ]:
map_hull(30, 30, 0, 'Boston MA', ['25', '26'], 0.3)

# Calgary AB

In [ ]:
explore_pts('Calgary')

In [ ]:
create_map(8, 5, 0, 'Calgary')

In [ ]:
map_hull(8, 5, 0, 'Calgary', ['121'], 0.9)

# Charlotte NC

In [ ]:
explore_pts('Charlotte NC')

In [ ]:
create_map(120, 50, 0, 'Charlotte NC')

In [ ]:
map_hull(120, 50, 0, 'Charlotte NC', ['10'], 0.9)

# Chicago IL

In [ ]:
explore_pts('Chicago IL')

In [ ]:
create_map(110, 50, 0, 'Chicago IL')

In [ ]:
map_hull(110, 50, 0, 'Chicago IL', ['10', '12', '13', '14', '15', '16'], 0.9)

# Cincinnati OH

In [ ]:
explore_pts('Cincinnati OH')

In [ ]:
create_map(10, 10, 0, 'Cincinnati OH')

In [ ]:
map_hull(10, 10, 0, 'Cincinnati OH', ['36'], 1)

# Cleveland OH

In [ ]:
explore_pts('Cleveland OH')

In [ ]:
create_map(30, 25, 0, 'Cleveland OH')

In [ ]:
map_hull(30, 25, 0, 'Cleveland OH', ['14'], .4)

# Colorado Springs CO

In [ ]:
explore_pts('Colorado Springs CO')

In [ ]:
create_map(10, 10, 0, 'Colorado Springs CO')

In [ ]:
map_hull(10, 10, 0, 'Colorado Springs CO', ['21'], .9)

# Columbus OH

In [ ]:
explore_pts('Columbus OH')

In [ ]:
create_map(30, 20, 0.03, 'Columbus OH')

In [ ]:
map_hull(30, 20, 0.03, 'Columbus OH', ['31'], .5)

# Dallas TX

In [ ]:
explore_pts('Dallas TX')

In [ ]:
create_map(75, 70, 0, 'Dallas TX')

In [ ]:
map_hull(75, 70, 0, 'Dallas TX', ['18', '19', '20', '21'], .9)

# Denver CO

In [ ]:
explore_pts('Denver CO')

In [ ]:
create_map(80, 80, 0, 'Denver CO')

In [ ]:
map_hull(80, 80, 0, 'Denver CO', ['9'], .8)

# Detroit MI

In [ ]:
explore_pts('Detroit MI')

In [ ]:
create_map(50, 40, 0.06, 'Detroit MI')

In [ ]:
map_hull(50, 40, 0.06, 'Detroit MI', ['7', '8'], .9)

# Edmonton AB

In [ ]:
explore_pts('Edmonton')

In [ ]:
create_map(50, 50, 0, 'Edmonton')

In [ ]:
map_hull(50, 50, 0, 'Edmonton', ['9'], .9)

# El Paso TX

In [ ]:
explore_pts('El Paso TX')

In [ ]:
create_map(25, 25, 0, 'El Paso TX')

In [ ]:
map_hull(25, 25, 0, 'El Paso TX', ['8'], .9)

# Fort Worth TX

In [ ]:
explore_pts('Fort Worth TX')

In [ ]:
create_map(12, 10, 0.02, 'Fort Worth TX')

In [ ]:
map_hull(12, 10, 0.02, 'Fort Worth TX', ['69', '71'], .9)

# Fresno CA

In [ ]:
explore_pts('Fresno CA')

In [ ]:
create_map(15, 15, 0, 'Fresno CA')

In [ ]:
map_hull(15, 15, 0, 'Fresno CA', ['14'], .9)

# Halifax NS

In [ ]:
explore_pts('Halifax')

In [ ]:
create_map(8, 5, 0, 'Halifax')

In [ ]:
map_hull(8, 5, 0, 'Halifax', ['40', '41', '45'], .7)

# Honolulu HI

In [ ]:
explore_pts('Urban Honolulu HI')

In [ ]:
create_map(50, 25, 0, 'Urban Honolulu HI')

In [ ]:
map_hull(50, 25, 0, 'Urban Honolulu HI', ['5'], .7)

# Houston TX

In [ ]:
explore_pts('Houston TX')

In [ ]:
create_map(125, 50, 0, 'Houston TX')

In [ ]:
map_hull(125, 50, 0, 'Houston TX', ['27', '28'], .9)

# Indianapolis IN

In [ ]:
explore_pts('Indianapolis city (balance) IN')

In [ ]:
create_map(50, 50, 0, 'Indianapolis city (balance) IN')

In [ ]:
map_hull(50, 50, 0, 'Indianapolis city (balance) IN', ['17'], .8)

# Jacksonville FL

In [ ]:
explore_pts('Jacksonville FL')

In [ ]:
create_map(50, 50, 0, 'Jacksonville FL')

In [ ]:
map_hull(50, 50, 0, 'Jacksonville FL', ['16'], .8)

# Kansas City MO

In [ ]:
explore_pts('Kansas City MO')

In [ ]:
create_map(60, 15, 0, 'Kansas City MO')

In [ ]:
map_hull(60, 15, 0, 'Kansas City MO', ['12', '13'], .9)

# Las Vegas NV

In [ ]:
explore_pts('Las Vegas NV')

In [ ]:
create_map(40, 20, 0, 'Las Vegas NV')

In [ ]:
map_hull(40, 20, 0, 'Las Vegas NV', ['8'], .8)

# London ON

In [ ]:
explore_pts('London')

In [ ]:
create_map(12, 10, 0, 'London')

In [ ]:
map_hull(12, 10, 0, 'London', ['22'], .8)

# Los Angeles CA

In [ ]:
explore_pts('Los Angeles CA')

In [ ]:
create_map(100, 100, 0.05, 'Los Angeles CA')

In [ ]:
map_hull(100, 100, 0.05, 'Los Angeles CA', ['21', '22'], .8)

# Louisville KY

In [ ]:
explore_pts('Louisville KY')

In [ ]:
create_map(20, 20, 0, 'Louisville KY')

In [ ]:
map_hull(20, 20, 0, 'Louisville KY', ['14', '15'], .8)

# Memphis TN

In [ ]:
explore_pts('Memphis TN')

In [ ]:
create_map(25, 25, 0, 'Memphis TN')

In [ ]:
map_hull(25, 25, 0, 'Memphis TN', ['15', '16'], .8)

# Miami FL

In [ ]:
explore_pts('Miami FL')

In [ ]:
create_map(50, 50, 0, 'Miami FL')

In [ ]:
map_hull(50, 50, 0, 'Miami FL', ['6', '9'], .8)

# Milwaukee WI

In [ ]:
explore_pts('Milwaukee WI')

In [ ]:
create_map(25, 10, 0.01, 'Milwaukee WI')

In [ ]:
map_hull(25, 10, 0.01, 'Milwaukee WI', ['28', '29', '30', '31'], .4)

# Minneapolis MN

In [ ]:
explore_pts('Minneapolis MN')

In [ ]:
create_map(25, 25, 0, 'Minneapolis MN')

In [ ]:
map_hull(25, 25, 0, 'Minneapolis MN', ['17'], .8)

# Mississauga ON

In [ ]:
explore_pts('Toronto')

In [ ]:
create_map(15, 15, 0, 'Toronto')

In [ ]:
map_hull(15, 15, 0, 'Toronto', ['36'], .8)

# Montreal QC

In [ ]:
explore_pts('Montréal')

In [ ]:
create_map(20, 20, 0, 'Montréal')

In [ ]:
map_hull(20, 20, 0, 'Montréal', ['93'], .3)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)

# ?

In [ ]:
explore_pts('')

In [ ]:
create_map(50, 50, 0, '')

In [ ]:
map_hull(50, 50, 0, '', [''], .8)